데이터 불러오기

In [1]:
# 훈련데이터
import time
start = time.time()

import cv2
import numpy as np
import pandas as pd

ID_List = pd.read_table('/Users/heogeonhyeock/Downloads/inha_data/inha_data/ID_List.txt',
                       sep = " ",
                       names = ["name", "path"])

train_img = []
train_name = []



for index in range(len(ID_List)):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/" + ID_List['path'][index]
    file = cv2.imread(A,0) # 회색
    file = cv2.resize(file, (50, 50)) # 이미지 크기 변경
    file = file.reshape(1,-1)[0] # 한 행으로 변경
    train_img.append(file) # 리스트에 이미지 추가
    train_name.append(ID_List['name'][index])

train_img = np.array(train_img)
train_name = np.array(train_name)

train = dict(data = train_img,
            target = train_name)

end = time.time()
print(end - start)

281.53580379486084


모델

In [2]:
# 데이터 분할

from sklearn.model_selection import train_test_split

Xtrain, Xvaild, Ytrain, Yvaild = train_test_split(train['data'],
                                                train['target'], 
                                                test_size = 1/3,  stratify=train['target'])

In [3]:
# 모델 생성

from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components = 150, whiten=True)
svc = SVC(kernel = "rbf", class_weight = "balanced", C=5, gamma = 0.005)
model = make_pipeline(pca,svc)

In [ ]:
# 하이퍼 파라미터 튜닝

"""
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__C' : [5],
              'svc__gamma' : [0.005]}

grid = GridSearchCV(model, param_grid, cv=2, n_jobs=-1)
grid.fit(Xtrain, ytrain)

model = grid.best_estimator_
"""

model.fit(Xtrain, Ytrain)

In [ ]:
# 모델 저장
import joblib
joblib.dump(model, './Inha.pkl')

모델 적용

In [ ]:
# 테스트 데이터

left_test_img = []
right_test_img = []

for index in range(6000):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/left_face_" + str(index) + ".jpg"
    B = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/right_face_" + str(index) + ".jpg" 
    
    file_left = cv2.imread(A,0) # 음영처리
    file_left = cv2.resize(file_left, (50, 50)) # 이미지 크기 변경
    file_left = file_left.reshape(1,-1)[0] # 한 행으로 변경
    
    file_right = cv2.imread(B,0)
    file_right = cv2.resize(file_right, (50,50))
    file_right = file_right.reshape(1,-1)[0]
    
    left_test_img.append(file_left)
    right_test_img.append(file_right)


In [ ]:
# 정답 저장

import os

predict = model.predict((left_test_img))

submission = pd.read_csv("/Users/heogeonhyeock/Downloads/inha_data/inha_data/sample_submission.csv")
submission["answer"] = predict[:6000]

path = '/Users/heogeonhyeock/Downloads/inha_data/inha_data'
os.chdir(path)
submission.to_csv("submission.csv", index = False)

submission